In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time 
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException,ElementClickInterceptedException, StaleElementReferenceException
import json
phone_url = [
    'https://www.thegioididong.com/dtdd-samsung',
    'https://www.thegioididong.com/dtdd-apple-iphone',
    'https://www.thegioididong.com/dtdd-oppo',
    'https://www.thegioididong.com/dtdd-xiaomi',
    'https://www.thegioididong.com/dtdd-vivo',
    'https://www.thegioididong.com/dtdd-realme',
    'https://www.thegioididong.com/dtdd-honor',
    'https://www.thegioididong.com/dtdd-apple-iphone-17-series',
    'https://www.thegioididong.com/dtdd-nokia',
    'https://www.thegioididong.com/dtdd-tecno'
]
options = Options()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080") # Đặt kích thước cửa sổ ảo

product_information = []
base_url = 'https://www.thegioididong.com'

for url in phone_url:
    Drive_path = '../chromedriver.exe'
    s = Service(executable_path=Drive_path)
    driver = webdriver.Chrome(service=s, options=options)
    
    driver.get(url)
    time.sleep(2)
    VIEW_MORE_CSS = ".view-more"   # dùng CSS selector cho chắc
    PRODUCT_ITEM_CSS = "ul.list.product li.item"  # selector cho các item

    # 1. Ẩn nhanh popup/chat (nếu có) để không che nút
    driver.execute_script("""
    var el = document.querySelector('.survey-wrap, .chat-widget, .sticky-chat');
    if(el) el.style.display = 'none';
    """)

    # 2. Lặp để click "Xem thêm" cho tới khi không còn
    prev_count = len(driver.find_elements(By.CSS_SELECTOR, PRODUCT_ITEM_CSS))

    while True:
        try:
            # đợi nút xuất hiện và có thể click
            view_more_button = WebDriverWait(driver, 8).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, VIEW_MORE_CSS))
            )

            # scroll phần tử vào giữa màn hình
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", view_more_button)
            time.sleep(0.5)

            try:
                view_more_button.click()   # ưu tiên click bình thường
            except (ElementClickInterceptedException, StaleElementReferenceException):
                # fallback dùng JS click nếu bị chặn hoặc stale
                try:
                    driver.execute_script("arguments[0].click();", view_more_button)
                except Exception as e:
                    print("Không click được bằng JS:", e)
                    break

            # chờ item mới được load (số lượng item tăng)
            WebDriverWait(driver, 7).until(
                lambda d: len(d.find_elements(By.CSS_SELECTOR, PRODUCT_ITEM_CSS)) > prev_count
            )
            prev_count = len(driver.find_elements(By.CSS_SELECTOR, PRODUCT_ITEM_CSS))
            time.sleep(0.5)

        except TimeoutException:
            # không tìm thấy nút hoặc không có thêm sản phẩm => thoát vòng
            print("Không còn nút 'Xem thêm' hoặc timeout, dừng click.")
            break
        except Exception as e:
            print("Gặp lỗi, dừng click:", type(e).__name__, e)
            break

    html_content = driver.page_source
    driver.quit()
    
    print(f"Processing: {url}")
    try:
        soup = BeautifulSoup(html_content, 'html.parser')
        product_list_container = soup.find('ul', class_='listproduct')
        
        all_products = product_list_container.select('li.item.ajaxed.__cate_42')
        print(f"Tìm thấy {len(all_products)} sản phẩm trên trang.")

        for item in all_products:
            product_dict = {}

            # Take link
            link = item.find('a', class_='main-contain')
            if link:
                product_dict['link'] = urljoin(base_url, link.get('href'))

            # Take name
            name_tag = item.find('h3')
            if name_tag:
                product_dict['name'] = name_tag.get('title')

            # Take price
            price_tag = item.find('strong', class_='price')
            if price_tag:
                product_dict['price'] = price_tag.text.strip()

            # Take item-gift
            gift_tag = item.find('p', class_='item-gift')
            if gift_tag:
                product_dict['gift'] = gift_tag.text.strip()

            # Take rating
            rating_tag = item.find('div', class_='rating_Compare has_compare has_quantity') 
            if rating_tag:

                star_tag = rating_tag.find('div', class_='vote-txt').find('b')
                if star_tag:
                    product_dict['start_rating'] = star_tag.text.strip()

                count_tag = rating_tag.find('span') 
                if count_tag:
                    review_text = count_tag.text.replace('•','').strip()   
                    product_dict['Sold_item'] = review_text

            if 'name' in product_dict:
                product_information.append(product_dict)

    except Exception as e:
        print(f"Lỗi khi thu thập link: {e}")
        product_links_list = [] 

Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-samsung
Tìm thấy 21 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-apple-iphone
Tìm thấy 14 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-oppo
Tìm thấy 29 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-xiaomi
Tìm thấy 19 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-vivo
Tìm thấy 16 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-realme
Tìm thấy 13 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc timeout, dừng click.
Processing: https://www.thegioididong.com/dtdd-honor
Tìm thấy 13 sản phẩm trên trang.
Không còn nút 'Xem thêm' hoặc ti

In [9]:
if len(product_information) > 0:
    df = pd.DataFrame(product_information)
    output_dir = '../data/raw/product_information.csv'
    df.to_csv(output_dir, index=False, encoding='utf-8-sig')

comment_links_list = []    
for info in product_information:
    comment_links_list.append(info['link'] + '/danh-gia')
print(comment_links_list[0])

https://www.thegioididong.com/dtdd/samsung-galaxy-s25-fe-5g-8gb-128gb/danh-gia


In [ ]:
Drive_path = '../chromedriver.exe'

s = Service(executable_path=Drive_path)

driver = webdriver.Chrome(service=s)

all_scraped_comment = []

for url in comment_links_list:
    product_name = url.split("/")[-2]
    print(f"Take data from: {product_name}")
    try:
        driver.get(url)
        time.sleep(2)
    except Exception:
        print(f"Bình luận quá ít")
        continue

    while True:
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")
        comments = soup.select("ul.comment-list li.par")
        
        print(f"Tìm thấy {len(comments)} bình luận ở trang này")

        for c in comments:
            user = c.find("p", class_="cmt-top-name")
            text = c.find("p", class_="cmt-txt")
            rating = len(c.find_all("i", class_="iconcmt-starbuy"))

            if user and text:
                all_scraped_comment.append({
                    "user": user.text.strip(),
                    "comment_text": text.text.strip(),
                    "rating": rating,
                    "page_source": product_name
                })
        try:
            # 1. Tìm phần tử active (đây là trang hiện tại)
            # Dựa vào cấu trúc web, trang active là <span class="active">
            active_element = driver.find_element(By.CSS_SELECTOR, "div.pagcomment span.active")
            
            # 2. Tìm nút bấm của trang kế tiếp
            # Đây là thẻ <a> đầu tiên nằm "ngay sau" (following-sibling) thẻ active
            # Nếu không có thẻ <a> nào (vì đang ở trang cuối), nó sẽ văng lỗi NoSuchElementException
            next_button = active_element.find_element(By.XPATH, "following-sibling::a[1]")
            
            # Lấy text của trang kế tiếp (ví dụ: "4") để dùng cho việc 'wait'
            # Chúng ta cần text này để đợi cho đến khi "4" trở thành active
            next_page_text = next_button.text.strip().replace("'", "") # Dùng replace phòng trường hợp có ký tự lạ

            # 3. Click vào nút kế tiếp
            driver.execute_script("arguments[0].click();", next_button)

            # 4. Chờ cho đến khi trang mới load xong
            # Điều kiện chờ là: thẻ <span class="active"> MỚI phải có text là trang ta vừa click
            WebDriverWait(driver, 10).until(
                EC.text_to_be_present_in_element(
                    (By.CSS_SELECTOR, "div.pagcomment span.active"),
                    next_page_text
                )
            )

            time.sleep(1) # Chờ một chút để JS (nếu có) load

        except (TimeoutException, NoSuchElementException):
            # Lỗi sẽ xảy ra trong các trường hợp sau:
            # 1. NoSuchElementException: Không tìm thấy 'span.active' (trang không có pagination).
            # 2. NoSuchElementException: Không tìm thấy 'following-sibling::a[1]' (đã ở trang cuối cùng).
            # 3. TimeoutException: 'WebDriverWait' bị timeout (trang không load kịp).

            # Trong mọi trường hợp, đó là dấu hiệu để dừng cào trang này.
            print("Không còn trang kế tiếp hoặc đã hết thời gian chờ.")
            break

driver.quit()
print(f"Tổng số bình luận: {len(all_scraped_comment)}")
print(all_scraped_comment[:5])

Take data from: samsung-galaxy-s25-fe-5g-8gb-128gb
Tìm thấy 0 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-a17-5g-8gb-128gb
Tìm thấy 0 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-a07-4gb-64gb
Tìm thấy 0 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-s25-ultra
Tìm thấy 20 bình luận ở trang này
Tìm thấy 5 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-z-fold7
Tìm thấy 0 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-a56
Tìm thấy 20 bình luận ở trang này
Tìm thấy 3 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-a26-6gb-128gb
Tìm thấy 4 bình luận ở trang này
Không còn trang kế tiếp hoặc đã hết thời gian chờ.
Take data from: samsung-galaxy-a06-5g-6gb

In [11]:
if len(all_scraped_comment) > 0:
    df = pd.DataFrame(all_scraped_comment)
    output_path = "../data/raw/reviews.csv"
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"Stored successfully!")

Stored successfully!
